# 01 - Sistema de Recomendación: Filtrado Colaborativo

Este notebook implementa un sistema de recomendación basado en usuarios usando la librería `surprise`.

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Cargar dataset
df = pd.read_csv('../data/ratings.csv')
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Entrenar modelo
trainset, testset = train_test_split(data, test_size=0.25)
algo = KNNBasic(sim_options={'user_based': True})
algo.fit(trainset)

# Predecir y evaluar
predictions = algo.test(testset)
print('RMSE:', accuracy.rmse(predictions))

In [ ]:
# Recomendaciones personalizadas para un usuario
from collections import defaultdict

def get_top_n(predictions, n=3):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=3)
top_n